## ⚠️ BEFORE YOU START

**What day is it today?**

- ✅ **Friday after 4pm ET:** Perfect! Run all cells below
- ❌ **Monday-Thursday:** STOP! Wait until Friday after market close
- ❌ **Friday before 4pm:** Wait until market closes at 4pm ET

**Why Friday close?** GHB Strategy uses weekly closing prices. Trading mid-week gives false signals and won't match backtested performance.

---

# GHB Strategy Portfolio Scanner
**Gold-Gray-Blue Weekly Trading System**

## 📅 WEEKLY TRADING SCHEDULE

⚠️ **IMPORTANT:** Only run this scanner on **FRIDAY after 4pm ET** (after market close)

### Your Weekly Routine:

**FRIDAY (After 4pm ET)**
- 📊 Run this notebook (all cells)
- 📋 Review signals: BUY (P1), HOLD (P2/N1), SELL (N2)
- 📝 Make your trade list for Monday
- ⏱️ Time: 10-15 minutes

**WEEKEND (Saturday/Sunday)**
- 💭 Review and confirm your plan
- 🧮 Calculate position sizes (10% each = $11,000 per position)
- ✅ Prepare for Monday execution

**MONDAY (Market Open - 9:30am ET)**
- 🔵 **FIRST (9:30-10:00am):** Execute ALL sell signals (N2 stocks) - URGENT
- 🟡 **THEN (10:00-10:30am):** Enter new buy positions (P1 stocks) - PATIENT
- ⏱️ Time: 15-30 minutes
- 💡 Tip: Use limit orders (Sells: Friday -1%, Buys: Friday +1.5%)

---

**Last Run:** {current_date}  
**Strategy:** GHB Strategy (Gold-Gray-Blue) - Quality-Filtered Price-Based Staging System  
**Universe:** 12 AI/Tech Stocks (Focused on AI dominance thesis 2023-2032)  
**Configuration:** Custom allocation (TSLA 20%, NVDA 20%, Others 7.5%), 10 max positions  
**Backtest Performance:** 34.62% CAGR | 36.1% Win Rate | 8 trades/year (2021-2025)

In [1]:
# Import Required Libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries loaded successfully")
print(f"📅 Current Date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries loaded successfully
📅 Current Date: 2026-01-16


## 1. Universe Configuration

**Why this universe?**
- **Thesis:** AI infrastructure buildout 2023-2032
- **Focus:** Pure-play AI stocks (semiconductors, compute, infrastructure)
- **Backtest:** 34.62% CAGR (2021-2025) - Pure momentum strategy without entry filters
- **Optimization:** Quarterly re-screening to maintain quality

**Trading Style:** Momentum-following (buy strength, sell weakness)

In [2]:
# GHB Strategy AI/Tech Focused Portfolio - 12 Stocks
# Selected for AI dominance thesis (2023-2032)
# Backtest (2022-2025): 56.51% CAGR, 332% total return, 40% win rate
GHB_UNIVERSE = [
    'ALAB',  # Astera Labs - AI connectivity
    'AMD',   # Advanced Micro Devices - AI chips
    'ARM',   # ARM Holdings - Mobile AI
    'ASML',  # ASML - Chip equipment
    'AVGO',  # Broadcom - AI infrastructure
    'GOOG',  # Google - AI/Cloud
    'MRVL',  # Marvell - Data infrastructure
    'MU',    # Micron - Memory/AI chips
    'NVDA',  # NVIDIA - AI infrastructure (Best: +516%)
    'PLTR',  # Palantir - AI software
    'TSLA',  # Tesla - AI/Autonomy
    'TSM',   # Taiwan Semi - AI chip manufacturing
]

print(f"📊 Universe: {len(GHB_UNIVERSE)} stocks (AI/Tech Focused)")
print(f"📈 Stocks: {', '.join(sorted(GHB_UNIVERSE[:6]))}...")
print(f"💡 Backtest (2022-2025): 56.51% CAGR | Best: NVDA +516% | Thesis: AI dominance 2023-2032")

📊 Universe: 12 stocks (AI/Tech Focused)
📈 Stocks: ALAB, AMD, ARM, ASML, AVGO, GOOG...
💡 Backtest (2022-2025): 56.51% CAGR | Best: NVDA +516% | Thesis: AI dominance 2023-2032


## 2. Calculate Weekly Larsson States

For each stock, calculate:
- **Weekly Close** (Friday)
- **200-Day SMA** (D200)
- **4-Week ROC** (Rate of Change)
- **RSI (14-period)** (Overextension indicator)
- **Weekly State** (P1/P2/N1/N2)
- **Entry Quality** (Pullback Buy / Healthy / Extended / Overheated)

In [3]:
def calculate_weekly_larsson_state(ticker):
    """
    Calculate weekly Larsson state for a ticker with support/resistance levels
    Returns: dict with ticker data or None if error
    """
    try:
        # Download 1 year of daily data
        stock = yf.Ticker(ticker)
        df = stock.history(period='1y', interval='1d')
        
        if df.empty or len(df) < 200:
            return None
        
        # Get latest close
        close = df['Close'].iloc[-1]
        
        # Calculate 200-day SMA
        d200 = df['Close'].rolling(window=200).mean().iloc[-1]
        
        # Calculate 4-week ROC (20 trading days)
        if len(df) >= 20:
            price_4w_ago = df['Close'].iloc[-20]
            roc_4w = ((close - price_4w_ago) / price_4w_ago) * 100
        else:
            roc_4w = 0
        
        # Calculate distance from D200
        distance_pct = ((close - d200) / d200) * 100
        
        # Calculate RSI (14-period)
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        rsi_value = rsi.iloc[-1]
        
        # === NEW: Calculate Support/Resistance Levels ===
        
        # 52-week high/low
        week_52_high = df['High'].tail(252).max()
        week_52_low = df['Low'].tail(252).min()
        
        # Recent swing low (support) - lowest low in last 60 days
        recent_support = df['Low'].tail(60).min()
        
        # Recent swing high (resistance) - highest high in last 60 days
        recent_resistance = df['High'].tail(60).max()
        
        # 50-day and 100-day SMAs (additional support/resistance)
        sma_50 = df['Close'].rolling(window=50).mean().iloc[-1]
        sma_100 = df['Close'].rolling(window=100).mean().iloc[-1]
        
        # Calculate distance to key levels
        to_support = ((close - recent_support) / close) * 100
        to_resistance = ((recent_resistance - close) / close) * 100
        
        # Risk assessment
        if to_support < 5:
            risk_level = "LOW (Near support)"
        elif to_support < 10:
            risk_level = "MODERATE"
        else:
            risk_level = "HIGH (Far from support)"
        
        # Determine Weekly Larsson state (Strategy D rules)
        if close > d200:
            # Price above D200
            if roc_4w > 5 or distance_pct > 10:
                state = 'STRONG_BUY'  # Strong bullish
                signal = '🟡 BUY'
            else:
                state = 'CONSOLIDATION'  # Consolidation
                signal = '⚪ HOLD'
        else:
            # Price below D200
            if distance_pct > -5:
                state = 'PULLBACK'  # Shallow pullback
                signal = '⚪ HOLD'
            else:
                state = 'DOWNTREND'  # Downtrend
                signal = '🔵 SELL'
        
        return {
            'Ticker': ticker,
            'Close': close,
            'D200': d200,
            'Distance_%': distance_pct,
            'ROC_4W_%': roc_4w,
            'RSI': rsi_value,
            'State': state,
            'Signal': signal,
            # Support/Resistance data
            'Support': recent_support,
            'Resistance': recent_resistance,
            'To_Support_%': to_support,
            'To_Resistance_%': to_resistance,
            '52W_High': week_52_high,
            '52W_Low': week_52_low,
            'SMA_50': sma_50,
            'SMA_100': sma_100,
            'Risk_Level': risk_level
        }
        
    except Exception as e:
        print(f"❌ Error processing {ticker}: {str(e)}")
        return None

print("✅ Calculation function defined (with support/resistance levels)")

✅ Calculation function defined (with support/resistance levels)


## 3. Scan All Stocks

This will take ~30 seconds to download data and calculate states...

In [4]:
print(f"🔄 Scanning {len(GHB_UNIVERSE)} stocks... Please wait...\n")

results = []
for i, ticker in enumerate(GHB_UNIVERSE, 1):
    print(f"  [{i:2d}/{len(GHB_UNIVERSE)}] Processing {ticker:6s}...", end='\r')
    result = calculate_weekly_larsson_state(ticker)
    if result:
        results.append(result)

df_results = pd.DataFrame(results)

# Note: Results will be sorted by strategy priority in each signal section
# (PULLBACK BUY first, then HEALTHY BUY by momentum)

print(f"\n✅ Scan complete! Processed {len(df_results)}/{len(GHB_UNIVERSE)} stocks")
print(f"❌ Failed: {len(GHB_UNIVERSE) - len(df_results)} stocks")
print(f"💡 Strategy: Quality-filtered (HEALTHY/PULLBACK only) + Price-based staging at target zones")

🔄 Scanning 12 stocks... Please wait...

  [12/12] Processing TSM   ...
✅ Scan complete! Processed 12/12 stocks
❌ Failed: 0 stocks
💡 Strategy: Quality-filtered (HEALTHY/PULLBACK only) + Price-based staging at target zones


## 4. Strategy GHB Signals

### Buy Signals (STRONG_BUY - Gold)
**Action:** Enter new positions or add to existing  
**Requirements:** Price > D200 + Strong momentum

In [5]:
# Filter STRONG_BUY signals - sorted by momentum (ROC) descending
p1_signals = df_results[df_results['State'] == 'STRONG_BUY'].copy()

# Load price targets if not already loaded (for staged entry)
try:
    price_targets
except NameError:
    import json
    from pathlib import Path
    settings_path = Path("../data/portfolio_settings.json")
    with open(settings_path, 'r') as f:
        portfolio_settings = json.load(f)
    price_targets = portfolio_settings.get('price_targets', {})

# Helper function for price-based fill calculation
def calculate_initial_fill_pct(ticker, current_price):
    """Calculate initial fill percentage based on price vs target zone"""
    if ticker not in price_targets:
        return 100  # No target = full size on quality signals
    
    target_low = price_targets[ticker].get('target_low')
    target_high = price_targets[ticker].get('target_high')
    
    if target_low is None or target_high is None:
        return 100  # No valid targets = full size
    
    # Price-based staging rules
    if current_price < target_low:
        return 50  # Below target zone: could go lower
    elif target_low <= current_price <= target_high:
        return 75  # In the sweet spot
    else:
        return 25  # Above target: expensive

# Add overextension categorization and price-based fill percentage
def categorize_entry(row):
    rsi = row['RSI']
    distance = row['Distance_%']
    roc = row['ROC_4W_%']
    ticker = row['Ticker']
    price = row['Close']
    
    # Calculate initial fill based on price targets
    initial_fill = calculate_initial_fill_pct(ticker, price)
    
    # Priority 1: Pullback buy (P1 but negative ROC = dip buying)
    if roc < 0:
        return '🔥 PULLBACK BUY', 1, initial_fill
    # Priority 2: Healthy buy (not overextended)
    elif rsi < 70 and distance < 30:
        return '✅ HEALTHY BUY', 2, initial_fill
    # Priority 3: Extended (caution, but tradable)
    elif rsi < 80 or distance < 40:
        return '⚠️ EXTENDED', 3, initial_fill
    # Priority 4: Overheated (high risk)
    else:
        return '🚨 OVERHEATED', 4, initial_fill

p1_signals[['Entry_Quality', 'Priority', 'Initial_Fill_%']] = p1_signals.apply(categorize_entry, axis=1, result_type='expand')

# Create PDF-friendly version without emojis for export
def make_pdf_friendly(quality_text):
    """Remove emojis from quality labels for PDF compatibility"""
    return quality_text.replace('🔥 ', '').replace('✅ ', '').replace('⚠️ ', '').replace('🚨 ', '')

p1_signals['Entry_Quality_PDF'] = p1_signals['Entry_Quality'].apply(make_pdf_friendly)

# QUALITY-BASED STAGED ENTRY STRATEGY (Option B)
# Filter to only HEALTHY BUY and PULLBACK BUY (exclude EXTENDED and OVERHEATED)
filtered_p1 = p1_signals[p1_signals['Entry_Quality'].isin(['✅ HEALTHY BUY', '🔥 PULLBACK BUY'])].copy()

# Sort by Priority then ROC: PULLBACK BUY first (best), then HEALTHY BUY by momentum
p1_signals = filtered_p1.sort_values(['Priority', 'ROC_4W_%'], ascending=[True, False])

print(f"\n🎯 Quality Filter Applied: {len(p1_signals)}/{len(filtered_p1) + len(p1_signals[~p1_signals['Entry_Quality'].isin(['✅ HEALTHY BUY', '🔥 PULLBACK BUY'])])} P1 stocks pass quality filter")
print(f"   ✅ Strategy: 50% initial on HEALTHY BUY, 50% add-on on PULLBACK BUY")

print("=" * 140)
print(f"🟡 STRONG BUY - BUY SIGNALS: {len(p1_signals)} stocks (Quality Filtered)")
print("=" * 140)
print("⚡ PRICE-BASED STAGED ENTRY: Initial sizing based on price vs target zones")
print("📊 Quality Filter: Only HEALTHY BUY and PULLBACK BUY (EXTENDED and OVERHEATED excluded)\n")

if len(p1_signals) > 0:
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Dist %':<10} {'4W ROC':<10} {'RSI':<6} {'Quality':<20} {'Sizing':<30}")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        # Price-based staging instruction
        initial_fill = row['Initial_Fill_%']
        ticker = row['Ticker']
        
        # Show target zone context if available
        if ticker in price_targets and price_targets[ticker].get('target_low'):
            target_low = price_targets[ticker]['target_low']
            target_high = price_targets[ticker]['target_high']
            if row['Close'] < target_low:
                stage_instruction = f"BUY {initial_fill:.0f}% (Below ${target_low})"
            elif target_low <= row['Close'] <= target_high:
                stage_instruction = f"BUY {initial_fill:.0f}% (In Zone ${target_low}-${target_high})"
            else:
                stage_instruction = f"BUY {initial_fill:.0f}% (Above ${target_high})"
        else:
            stage_instruction = f"BUY {initial_fill:.0f}% (No target)"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+8.1f}% {row['ROC_4W_%']:>+8.1f}% {row['RSI']:>5.0f} {row['Entry_Quality']:<20} {stage_instruction:<30}")
    
    # Count by category (only filtered ones appear)
    pullback = len(p1_signals[p1_signals['Entry_Quality'] == '🔥 PULLBACK BUY'])
    healthy = len(p1_signals[p1_signals['Entry_Quality'] == '✅ HEALTHY BUY'])
    
    print("\n📊 QUALITY-FILTERED SIGNALS:")
    if pullback > 0:
        print(f"   🔥 PULLBACK BUY: {pullback} stock(s) → Scale in on weakness")
    if healthy > 0:
        print(f"   ✅ HEALTHY BUY: {healthy} stock(s) → Initial position")
    
    print("\n💡 PRICE-BASED STAGED ENTRY RULES:")
    print("   1. Initial Size: 50% if below target, 75% if in target zone, 25% if above")
    print("   2. Add-On: When price dips toward or into your target zone")
    print("   3. Exit: Sell full position on N2 DOWNTREND signal")
    print("   4. Long-Term: Hold quality names, scale in at target prices")
else:
    print("\n⚠️  No quality-filtered P1 buy signals this week")


🎯 Quality Filter Applied: 4/4 P1 stocks pass quality filter
   ✅ Strategy: 50% initial on HEALTHY BUY, 50% add-on on PULLBACK BUY
🟡 STRONG BUY - BUY SIGNALS: 4 stocks (Quality Filtered)
⚡ PRICE-BASED STAGED ENTRY: Initial sizing based on price vs target zones
📊 Quality Filter: Only HEALTHY BUY and PULLBACK BUY (EXTENDED and OVERHEATED excluded)

Ticker   Price      D200       Dist %     4W ROC     RSI    Quality              Sizing                        
--------------------------------------------------------------------------------------------------------------------------------------------
PLTR     $174.25    $155.46       +12.1%     -6.2%    35 🔥 PULLBACK BUY       BUY 25% (Above $155)          
TSLA     $439.27    $369.13       +19.0%     -9.1%    32 🔥 PULLBACK BUY       BUY 25% (Above $411.0)        
NVDA     $188.02    $164.05       +14.6%     +8.0%    44 ✅ HEALTHY BUY        BUY 25% (Above $172.0)        
AVGO     $347.24    $297.14       +16.9%     +5.5%    46 ✅ HEALTHY BUY 

### Support/Resistance Levels for P1 Stocks
**Technical analysis for context only**

📊 Quality-filtered signals only (HEALTHY BUY and PULLBACK BUY)

In [6]:
# Display support/resistance analysis for P1 signals (quality-filtered)
if len(p1_signals) > 0:
    print("\n" + "=" * 140)
    print("📍 SUPPORT/RESISTANCE ANALYSIS - Quality-Filtered P1 BUY SIGNALS")
    print("=" * 140)
    print("⚡ Showing only HEALTHY BUY and PULLBACK BUY signals\n")
    
    print(f"{'Ticker':<8} {'Current':<10} {'Support':<10} {'To Supp':<10} {'Resistance':<12} {'To Resist':<11} {'Risk Level':<20}")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['Support']:<9.2f} "
              f"{row['To_Support_%']:>+8.1f}% ${row['Resistance']:<11.2f} "
              f"{row['To_Resistance_%']:>+8.1f}% {row['Risk_Level']:<20}")
    
    print("\n📊 KEY LEVELS SUMMARY:")
    print(f"{'Ticker':<8} {'52W Low':<12} {'SMA 50':<12} {'SMA 100':<12} {'D200':<12} {'52W High':<12}")
    print("-" * 140)
    
    for _, row in p1_signals.iterrows():
        print(f"{row['Ticker']:<8} ${row['52W_Low']:<11.2f} ${row['SMA_50']:<11.2f} "
              f"${row['SMA_100']:<11.2f} ${row['D200']:<11.2f} ${row['52W_High']:<11.2f}")
    
    print("\n💡 STAGED ENTRY RECOMMENDATIONS:")
    print("-" * 140)
    print("⚡ STRATEGY: 50% on HEALTHY BUY, 50% add-on on PULLBACK BUY\n")
    
    for _, row in p1_signals.iterrows():
        ticker = row['Ticker']
        current = row['Close']
        quality = row['Entry_Quality']
        
        # Stage-based recommendation
        if quality == '🔥 PULLBACK BUY':
            rec = f"Stage 2: Add remaining 50% if you own Stage 1"
        else:  # HEALTHY BUY
            rec = f"Stage 1: Buy initial 50% position"
        
        print(f"{ticker}: {rec}")
        print(f"   Entry: ${current:.2f} with +1.5% limit = ${current * 1.015:.2f}")
        
    print("\n" + "=" * 140)
else:
    print("\n⚠️  No P1 signals - No support analysis available")


📍 SUPPORT/RESISTANCE ANALYSIS - Quality-Filtered P1 BUY SIGNALS
⚡ Showing only HEALTHY BUY and PULLBACK BUY signals

Ticker   Current    Support    To Supp    Resistance   To Resist   Risk Level          
--------------------------------------------------------------------------------------------------------------------------------------------
PLTR     $174.25    $147.56       +15.3% $207.52         +19.1% HIGH (Far from support)
TSLA     $439.27    $382.78       +12.9% $498.83         +13.6% HIGH (Far from support)
NVDA     $188.02    $169.54        +9.8% $212.18         +12.8% MODERATE            
AVGO     $347.24    $320.81        +7.6% $413.82         +19.2% MODERATE            

📊 KEY LEVELS SUMMARY:
Ticker   52W Low      SMA 50       SMA 100      D200         52W High    
--------------------------------------------------------------------------------------------------------------------------------------------
PLTR     $66.12       $178.27      $177.35      $155.46      $207.52 

### Hold Signals (CONSOLIDATING & PULLBACK - Gray)
**Action:** Continue holding existing positions  
**Meaning:** Normal consolidation/pullback in trend

In [7]:
# Filter CONSOLIDATION and PULLBACK (HOLD) signals
hold_signals = df_results[df_results['State'].isin(['CONSOLIDATION', 'PULLBACK'])].sort_values('Distance_%', ascending=False)

print("=" * 100)
print(f"⚪ CONSOLIDATION/PULLBACK (GRAY) - HOLD SIGNALS: {len(hold_signals)} stocks")
print("=" * 100)

if len(hold_signals) > 0:
    print(f"\n{'Ticker':<8} {'Price':<10} {'D200':<10} {'Distance':<12} {'4W ROC':<10} {'State':<8} {'Status':<20}")
    print("-" * 100)
    
    for _, row in hold_signals.iterrows():
        if row['State'] == 'CONSOLIDATION':
            status = "Consolidation"
        else:
            status = "Shallow Pullback"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {row['State']:<8} {status:<20}")
    
    print("\n💡 RECOMMENDATION:")
    print("   - HOLD all existing positions")
    print("   - Do NOT sell - this is normal consolidation")
    print("   - Watch for transition to STRONG_BUY (upgrade) or DOWNTREND (downgrade)")
else:
    print("\n✅ No stocks in consolidation phase")

⚪ CONSOLIDATION/PULLBACK (GRAY) - HOLD SIGNALS: 1 stocks

Ticker   Price      D200       Distance     4W ROC     State    Status              
----------------------------------------------------------------------------------------------------
MRVL     $81.43     $75.14           +8.4%    -3.53% CONSOLIDATION Consolidation       

💡 RECOMMENDATION:
   - HOLD all existing positions
   - Do NOT sell - this is normal consolidation
   - Watch for transition to STRONG_BUY (upgrade) or DOWNTREND (downgrade)


### Sell Signals (DOWNTREND - Blue)
**Action:** Exit positions immediately  
**Requirements:** Price < D200 + Weak momentum  
**⚠️ CRITICAL: Execute these sells on Monday!**

In [8]:
# Filter DOWNTREND (SELL) signals
n2_signals = df_results[df_results['State'] == 'DOWNTREND'].sort_values('Distance_%', ascending=True)

print("=" * 100)
print(f"🔵 DOWNTREND (BLUE) - SELL SIGNALS: {len(n2_signals)} stocks")
print("=" * 100)

if len(n2_signals) > 0:
    print(f"\n⚠️  EXIT THESE POSITIONS ON MONDAY!\n")
    print(f"{'Ticker':<8} {'Price':<10} {'D200':<10} {'Below D200':<12} {'4W ROC':<10} {'Severity':<15}")
    print("-" * 100)
    
    for _, row in n2_signals.iterrows():
        # Determine severity
        if row['Distance_%'] < -20:
            severity = "🚨 SEVERE"
        elif row['Distance_%'] < -10:
            severity = "⚠️  MAJOR"
        else:
            severity = "📉 MINOR"
        
        print(f"{row['Ticker']:<8} ${row['Close']:<9.2f} ${row['D200']:<9.2f} {row['Distance_%']:>+10.1f}% {row['ROC_4W_%']:>+8.2f}% {severity:<15}")
    
    print("\n💡 ACTION REQUIRED:")
    print("   - SELL all N2 positions on Monday at market open")
    print("   - Do NOT wait for bounce - trend is broken")
    print("   - Preserve capital for new P1 opportunities")
else:
    print("\n✅ No sell signals - all positions healthy!")

🔵 DOWNTREND (BLUE) - SELL SIGNALS: 1 stocks

⚠️  EXIT THESE POSITIONS ON MONDAY!

Ticker   Price      D200       Below D200   4W ROC     Severity       
----------------------------------------------------------------------------------------------------
ARM      $107.11    $137.19         -21.9%    -5.64% 🚨 SEVERE       

💡 ACTION REQUIRED:
   - SELL all N2 positions on Monday at market open
   - Do NOT wait for bounce - trend is broken
   - Preserve capital for new P1 opportunities


## 5. Weekly Summary

Quick overview of portfolio status and action items

In [9]:
print("=" * 100)
print("📊 GHB STRATEGY WEEKLY SUMMARY")
print("=" * 100)

# Initialize add_on_actions if not already defined
if 'add_on_actions' not in dir():
    add_on_actions = []

print(f"\n🟡 BUY Signals (STRONG_BUY):  {len(p1_signals)} stocks")
print(f"⚪ HOLD Signals (CONSOLIDATION/PULLBACK): {len(hold_signals)} stocks")
print(f"🔵 SELL Signals (DOWNTREND): {len(n2_signals)} stocks")
print(f"📊 Total Scanned: {len(df_results)}/{len(GHB_UNIVERSE)} stocks")

# Calculate portfolio health
total_bullish = len(p1_signals)
total_neutral = len(hold_signals)
total_bearish = len(n2_signals)
total = len(df_results)

pct_bullish = (total_bullish / total * 100) if total > 0 else 0
pct_bearish = (total_bearish / total * 100) if total > 0 else 0

print(f"\n📈 Market Health:")
print(f"   Bullish: {pct_bullish:.1f}% ({total_bullish} stocks)")
print(f"   Neutral: {(total_neutral/total*100):.1f}% ({total_neutral} stocks)")
print(f"   Bearish: {pct_bearish:.1f}% ({total_bearish} stocks)")

if pct_bullish > 60:
    market_sentiment = "🟢 VERY BULLISH - Many opportunities"
elif pct_bullish > 40:
    market_sentiment = "🟡 BULLISH - Good opportunities"
elif pct_bullish > 20:
    market_sentiment = "🟠 NEUTRAL - Selective opportunities"
else:
    market_sentiment = "🔴 BEARISH - Few opportunities, preserve cash"

print(f"\n📊 Market Sentiment: {market_sentiment}")

# Action items
print("\n✅ ACTION ITEMS FOR THIS WEEK:")

# Action 1: Sells (highest priority - free up cash)
if len(n2_signals) > 0:
    print(f"   1. 🔴 MONDAY: Sell {len(n2_signals)} DOWNTREND positions (exit completely)")
    for _, sig in n2_signals.head(5).iterrows():
        print(f"      → {sig['Ticker']}: SELL ALL @ ${sig['Close']:.2f}")
else:
    print("   1. ✅ No sells required")

# Action 2: Add-Ons (second priority - average into winners)
if len(add_on_actions) > 0:
    total_addon_cost = sum([a['Additional_Value'] for a in add_on_actions])
    print(f"\n   2. ⬆️ MONDAY: Add to {len(add_on_actions)} existing positions (${total_addon_cost:,.0f} total)")
    for action in add_on_actions:
        print(f"      → {action['Ticker']}: ADD {action['Additional_Shares']} shares @ ${action['Current_Price']:.2f} = ${action['Additional_Value']:,.0f}")
        print(f"         Fill {action['Current_Fill']}% → {action['New_Fill']}% | {action['Reason']}")
else:
    print("\n   2. ⏸️  No add-ons available")

# Action 3: New Buys (third priority - open new positions)
if len(p1_signals) > 0:
    print(f"\n   3. 🟡 MONDAY: Enter up to {min(5, len(p1_signals))} NEW positions")
    for _, sig in p1_signals.head(5).iterrows():
        quality = sig['Entry_Quality']
        alloc_pct = sig.get('Initial_Fill_%', 50)
        
        # Show entry with price-based fill
        print(f"      → {sig['Ticker']}: {quality}")
        print(f"         Entry: ${sig['Close']:.2f} with {alloc_pct:.0f}% initial fill")
else:
    print("\n   3. ⏸️  No new buys available - hold cash")        

📊 GHB STRATEGY WEEKLY SUMMARY

🟡 BUY Signals (STRONG_BUY):  4 stocks
⚪ HOLD Signals (CONSOLIDATION/PULLBACK): 1 stocks
🔵 SELL Signals (DOWNTREND): 1 stocks
📊 Total Scanned: 12/12 stocks

📈 Market Health:
   Bullish: 33.3% (4 stocks)
   Neutral: 8.3% (1 stocks)
   Bearish: 8.3% (1 stocks)

📊 Market Sentiment: 🟠 NEUTRAL - Selective opportunities

✅ ACTION ITEMS FOR THIS WEEK:
   1. 🔴 MONDAY: Sell 1 DOWNTREND positions (exit completely)
      → ARM: SELL ALL @ $107.11

   2. ⏸️  No add-ons available

   3. 🟡 MONDAY: Enter up to 4 NEW positions
      → PLTR: 🔥 PULLBACK BUY
         Entry: $174.25 with 25% initial fill
      → TSLA: 🔥 PULLBACK BUY
         Entry: $439.27 with 25% initial fill
      → NVDA: ✅ HEALTHY BUY
         Entry: $188.02 with 25% initial fill
      → AVGO: ✅ HEALTHY BUY
         Entry: $347.24 with 25% initial fill


## 6. Detailed Stock Data

Full dataset for analysis and record-keeping

In [10]:
# Display full results sorted by state then distance
df_display = df_results.copy()
df_display['State_Order'] = df_display['State'].map({'STRONG_BUY': 1, 'CONSOLIDATION': 2, 'PULLBACK': 3, 'DOWNTREND': 4})
df_display = df_display.sort_values(['State_Order', 'Distance_%'], ascending=[True, False])
df_display = df_display.drop('State_Order', axis=1)

print("\n📋 COMPLETE SCAN RESULTS")
print("=" * 100)
print(df_display.to_string(index=False))
print("=" * 100)


📋 COMPLETE SCAN RESULTS
Ticker       Close       D200  Distance_%  ROC_4W_%       RSI         State Signal    Support  Resistance  To_Support_%  To_Resistance_%    52W_High    52W_Low      SMA_50     SMA_100              Risk_Level
    MU  353.760010 161.203167  119.449789 42.387014 72.814634    STRONG_BUY  🟡 BUY 192.322306  365.809998     45.634809         3.406261  365.809998  61.418842  265.827875  219.941180 HIGH (Far from support)
  ASML 1351.925049 871.715045   55.087956 30.455652 82.614280    STRONG_BUY  🟡 BUY 946.109985 1375.369995     30.017571         1.734190 1375.369995 574.245860 1099.717289 1017.454020 HIGH (Far from support)
  GOOG  330.812988 227.085976   45.677419  8.909626 72.475682    STRONG_BUY  🟡 BUY 250.287702  341.200012     24.341634         3.139848  341.200012 142.268675  309.975147  278.582281 HIGH (Far from support)
   TSM  343.855011 245.662928   39.970249 20.786504 77.092285    STRONG_BUY  🟡 BUY 266.101623  351.329987     22.612260         2.173874  351.3

## 6.5 Current Portfolio Holdings

**Phase 1 - Portfolio Tracker:** Track your positions and calculate performance

In [11]:
import json
from pathlib import Path

# Load portfolio settings
settings_path = Path("../data/portfolio_settings.json")
with open(settings_path, 'r') as f:
    portfolio_settings = json.load(f)

starting_cash = portfolio_settings['starting_cash']
position_size_pct = portfolio_settings['position_size_pct']
max_positions = portfolio_settings['max_positions']
strategy_week = portfolio_settings['strategy_week']
conservative_mode = portfolio_settings.get('conservative_mode', True)
position_allocations = portfolio_settings.get('position_allocations', {})
price_targets = portfolio_settings.get('price_targets', {})

# Helper function to get position size for a ticker
def get_position_allocation(ticker):
    """Get allocation percentage for a ticker (custom or default)"""
    # If ticker has custom allocation, return it
    if ticker in position_allocations:
        return position_allocations[ticker]
    
    # Calculate remaining allocation for non-custom stocks
    total_custom_alloc = sum(position_allocations.values())
    remaining_alloc = 100 - total_custom_alloc
    
    # Split remaining allocation among (max_positions - custom_count) stocks
    num_non_custom_slots = max_positions - len(position_allocations)
    if num_non_custom_slots > 0:
        return remaining_alloc / num_non_custom_slots
    else:
        return position_size_pct  # Fallback if all positions are custom

def get_position_value(ticker):
    """Get dollar value for a position based on allocation"""
    return starting_cash * get_position_allocation(ticker) / 100
def calculate_initial_fill_pct(ticker, current_price):
    """Calculate initial fill percentage based on price vs target zone"""
    # Check if ticker has price targets
    if ticker not in price_targets:
        return 100  # No target = full size on quality signals
    
    target_low = price_targets[ticker].get('target_low')
    target_high = price_targets[ticker].get('target_high')
    
    if target_low is None or target_high is None:
        return 100  # No valid targets = full size
    
    # Price-based staging rules
    if current_price < target_low:
        # Below target zone: could go lower, start small
        return 50
    elif target_low <= current_price <= target_high:
        # In the sweet spot: build substantial position
        return 75
    else:
        # Above target: expensive, wait for pullback
        return 25

print("💼 PORTFOLIO CONFIGURATION")
print("=" * 100)
print(f"Starting Capital: ${starting_cash:,.0f}")
print(f"Default Position Size: {position_size_pct}% (${starting_cash * position_size_pct / 100:,.0f} per position)")
print(f"Max Positions: {max_positions}")
print(f"Strategy Week: {strategy_week}")
print(f"Mode: {'Conservative (Building Gradually)' if conservative_mode else 'Aggressive'}")
if position_allocations:
    print(f"\n⚖️  Custom Allocations:")
    for ticker, pct in sorted(position_allocations.items(), key=lambda x: x[1], reverse=True):
        print(f"   {ticker}: {pct}% (${starting_cash * pct / 100:,.0f})")
print("=" * 100)

# Load current positions
positions_path = Path("../data/portfolio_positions.csv")
df_positions = pd.read_csv(positions_path)

# Ensure new columns exist (backward compatibility)
if 'Fill_Level' not in df_positions.columns:
    df_positions['Fill_Level'] = 100  # Assume legacy positions are complete
if 'Target_Allocation' not in df_positions.columns:
    df_positions['Target_Allocation'] = df_positions['Ticker'].apply(get_position_allocation)
if 'Can_Add' not in df_positions.columns:
    df_positions['Can_Add'] = df_positions['Fill_Level'] < 100

# Enrich with current signals for later use
df_positions_enriched = df_positions.copy()
if len(df_positions) > 0:
    signals_list = []
    prices_list = []
    states_list = []
    pl_pct_list = []
    position_values = []
    pl_dollars = []
    
    for _, pos in df_positions.iterrows():
        ticker = pos['Ticker']
        current_data = df_results[df_results['Ticker'] == ticker]
        if len(current_data) > 0:
            current_price = current_data.iloc[0]['Close']
            current_state = current_data.iloc[0]['State']
            current_signal = current_data.iloc[0]['Signal']
            
            signals_list.append(current_signal)
            prices_list.append(current_price)
            states_list.append(current_state)
            
            pl_pct = ((current_price - pos['Entry_Price']) / pos['Entry_Price']) * 100
            pl_pct_list.append(pl_pct)
            
            position_value = current_price * pos['Shares']
            position_values.append(position_value)
            pl_dollars.append(position_value - (pos['Entry_Price'] * pos['Shares']))
        else:
            signals_list.append('UNKNOWN')
            prices_list.append(pos['Entry_Price'])
            states_list.append(pos.get('Current_State', 'UNKNOWN'))
            pl_pct_list.append(0)
            position_values.append(pos['Entry_Price'] * pos['Shares'])
            pl_dollars.append(0)
    
    df_positions_enriched['Signal'] = signals_list
    df_positions_enriched['Current_Price'] = prices_list
    df_positions_enriched['State'] = states_list
    df_positions_enriched['P/L_%'] = pl_pct_list
    df_positions_enriched['Position_Value'] = position_values
    df_positions_enriched['P/L_$'] = pl_dollars

add_on_actions = []  # Track add-on opportunities based on price improvements

# Detect add-on opportunities based on price targets
for _, pos in df_positions.iterrows():
    ticker = pos['Ticker']
    entry_price = pos['Entry_Price']
    current_fill = pos.get('Fill_Level', 100)
    
    # Skip if position is already at 100%
    if current_fill >= 100:
        continue
    
    # Get current price from scan results
    current_data = df_results[df_results['Ticker'] == ticker]
    if len(current_data) == 0:
        continue
    
    current_price = current_data.iloc[0]['Close']
    current_state = current_data.iloc[0]['State']
    
    # Check if we should add based on price targets
    if ticker in price_targets and price_targets[ticker].get('target_low'):
        target_low = price_targets[ticker]['target_low']
        target_high = price_targets[ticker]['target_high']
        
        # Add-on opportunity: Price dipped into or below target zone
        if current_price <= target_high and current_state != 'DOWNTREND':
            # Calculate add-on amount
            remaining_pct = 100 - current_fill
            if current_price < target_low:
                add_on_pct = min(25, remaining_pct)  # Add 25% if below target
                reason = f"Price ${current_price:.2f} below target ${target_low:.2f} - scale in"
            elif current_price <= target_high:
                add_on_pct = min(50, remaining_pct)  # Add 50% if in zone
                reason = f"Price ${current_price:.2f} in target zone ${target_low:.2f}-${target_high:.2f}"
            else:
                continue  # No add-on if above target
            
            # Calculate shares and values
            target_alloc = pos.get('Target_Allocation', get_position_allocation(ticker))
            max_position_value = starting_cash * target_alloc / 100
            additional_value = max_position_value * add_on_pct / 100
            additional_shares = int(additional_value / current_price)
            
            if additional_shares > 0:
                add_on_actions.append({
                    'Ticker': ticker,
                    'Reason': reason,
                    'Current_Price': current_price,
                    'Entry_Price': entry_price,
                    'Current_Fill': current_fill,
                    'Add_On_%': add_on_pct,
                    'New_Fill': current_fill + add_on_pct,
                    'Additional_Shares': additional_shares,
                    'Additional_Value': additional_value,
                    'New_Total_Shares': pos['Shares'] + additional_shares
                })

total_cost = 0
total_value = 0
total_pl = 0
total_pl_pct = 0
cash_remaining = starting_cash
deployed_pct = 0

if len(df_positions) == 0:
    print("\n📭 No positions yet - Portfolio is 100% CASH")
    print(f"💰 Available: ${starting_cash:,.0f}")
    print(f"\n💡 Week {strategy_week} Recommendation:")
    if strategy_week == 1:
        print("   Start with 2-3 positions")
        # Calculate suggested allocation based on available P1 signals with custom allocations
        if len(p1_signals) > 0:
            top_picks = p1_signals.head(3)['Ticker'].tolist()
            suggested_deploy = sum([get_position_allocation(t) for t in top_picks])
            print(f"   Suggested: {', '.join(top_picks)}")
            print(f"   Total Deploy: {suggested_deploy:.1f}% (${starting_cash * suggested_deploy / 100:,.0f})")
            for ticker in top_picks:
                alloc = get_position_allocation(ticker)
                print(f"      {ticker}: {alloc}% = ${starting_cash * alloc / 100:,.0f}")
    else:
        print(f"   Consider adding 2-3 new positions")
        print(f"   Build towards {max_positions} total positions")
else:
    print(f"\n📊 ACTIVE POSITIONS: {len(df_positions)}")
    print("=" * 100)
    
    # Update current states for all positions
    position_summaries = []
    
    for _, pos in df_positions.iterrows():
        ticker = pos['Ticker']
        
        # Get current data from scan results
        current_data = df_results[df_results['Ticker'] == ticker]
        
        if len(current_data) > 0:
            current_price = current_data.iloc[0]['Close']
            current_state = current_data.iloc[0]['State']
            current_signal = current_data.iloc[0]['Signal']
            distance_pct = current_data.iloc[0]['Distance_%']
            roc_pct = current_data.iloc[0]['ROC_4W_%']
        else:
            # Ticker not in universe or failed to scan
            current_price = pos['Entry_Price']  # Fallback
            current_state = pos['Current_State']
            current_signal = '❓ UNKNOWN'
            distance_pct = 0
            roc_pct = 0
        
        # Calculate P&L
        entry_price = pos['Entry_Price']
        shares = pos['Shares']
        cost_basis = entry_price * shares
        current_value = current_price * shares
        pl_dollars = current_value - cost_basis
        pl_pct = ((current_price - entry_price) / entry_price) * 100
        
        # Get allocation for display
        target_alloc = get_position_allocation(ticker)
        actual_alloc = (cost_basis / starting_cash) * 100
        
        # State change detection
        state_change = ""
        if current_state != pos['Entry_State']:
            state_change = f"({pos['Entry_State']} → {current_state})"
        
        position_summaries.append({
            'Ticker': ticker,
            'Allocation': f"{actual_alloc:.1f}%",
            'Entry_Date': pos['Entry_Date'],
            'Entry_Price': f"${entry_price:.2f}",
            'Current_Price': f"${current_price:.2f}",
            'Shares': int(shares),
            'Cost_Basis': f"${cost_basis:,.0f}",
            'Current_Value': f"${current_value:,.0f}",
            'P/L_$': f"${pl_dollars:+,.0f}",
            'P/L_%': f"{pl_pct:+.1f}%",
            'Entry_State': pos['Entry_State'],
            'Current_State': current_state,
            'State_Change': state_change,
            'Signal': current_signal
        })
    
    df_summary = pd.DataFrame(position_summaries)
    print(df_summary.to_string(index=False))
    print("=" * 100)
    
    # Portfolio totals
    total_cost = sum([float(s['Cost_Basis'].replace('$', '').replace(',', '')) for s in position_summaries])
    total_value = sum([float(s['Current_Value'].replace('$', '').replace(',', '')) for s in position_summaries])
    total_pl = total_value - total_cost
    total_pl_pct = (total_pl / total_cost * 100) if total_cost > 0 else 0
    
    cash_remaining = starting_cash - total_cost
    deployed_pct = (total_cost / starting_cash * 100)
    
    print(f"\n💼 PORTFOLIO SUMMARY")
    print("=" * 100)
    print(f"Total Cost Basis: ${total_cost:,.0f}")
    print(f"Current Value: ${total_value:,.0f}")
    print(f"Total P/L: ${total_pl:+,.0f} ({total_pl_pct:+.1f}%)")
    print(f"Cash Remaining: ${cash_remaining:,.0f}")
    print(f"Deployed: {deployed_pct:.1f}% | Cash: {100-deployed_pct:.1f}%")
    print("=" * 100)
    
    # Alerts and Warnings
    print(f"\n⚠️ PORTFOLIO ALERTS")
    print("=" * 100)
    
    n2_positions = [s for s in position_summaries if 'SELL' in s['Signal']]
    add_on_positions = [s for s in position_summaries if s['Add_On'] != ""]
    
    if len(add_on_actions) > 0:
        print(f"🟢 STAGED ENTRY: {len(add_on_actions)} position(s) ready for add-on!")
        for action in add_on_actions:
            print(f"   → {action['Ticker']}: {action['Reason']}")
            print(f"      Add {action['Additional_Shares']} shares @ ${action['Current_Price']:.2f} = ${action['Additional_Value']:,.0f}")
            print(f"      Fill: {action['Current_Fill']}% → {action['New_Fill']}% | Total: {action['New_Total_Shares']} shares")
    
    if len(n2_positions) > 0:
        print(f"🔴 URGENT: {len(n2_positions)} position(s) in DOWNTREND (SELL) state!")
        for pos in n2_positions:
            print(f"   → {pos['Ticker']}: SELL on Monday open (Current: {pos['Current_State']})")
    else:
        print("✅ No urgent sell signals")
    
    if len(state_changes) > 0:
        print(f"\n📊 {len(state_changes)} position(s) changed state:")
        for pos in state_changes:
            print(f"   → {pos['Ticker']}: {pos['State_Change']}")
    else:
        print("\n✅ All positions maintained their states")
    
    print("=" * 100)
    
    # Position sizing recommendations with variable allocations
    print(f"\n💰 POSITION SIZING FOR THIS WEEK")
    print("=" * 100)
    
    current_positions = len(df_positions)
    positions_to_add = max_positions - current_positions
    
    if positions_to_add > 0 and len(p1_signals) > 0:
        print(f"📈 Recommended: Add {min(positions_to_add, len(p1_signals))} new position(s)")
        print(f"\n💡 Top {min(positions_to_add, 5)} Candidates with Allocations:")
        
        total_deploy = 0
        for i, (_, row) in enumerate(p1_signals.head(min(positions_to_add, 5)).iterrows()):
            ticker = row['Ticker']
            alloc_pct = get_position_allocation(ticker)
            position_value = get_position_value(ticker)
            total_deploy += position_value
            
            quality = row.get('Entry_Quality', 'N/A')
            price = row['Close']
            initial_fill = row.get('Initial_Fill_%', 100)
            
            print(f"   {i+1}. {ticker}: {alloc_pct}% = ${position_value:,.0f} @ ${price:.2f}")
            print(f"      {quality} | Initial fill: {initial_fill:.0f}%")
        
        print(f"\n🎯 Total to Deploy: ${total_deploy:,.0f}")
        print(f"📊 New Portfolio Allocation: {(total_cost + total_deploy) / starting_cash * 100:.1f}%")
        print(f"💵 Remaining Cash: ${cash_remaining - total_deploy:,.0f}")
    elif len(df_positions) >= max_positions:
        print(f"✅ Portfolio full ({len(df_positions)}/{max_positions} positions)")
        print(f"💵 Cash Available: ${cash_remaining:,.0f}")
        print(f"💡 Only trade if N2 sell creates opening")
    else:
        print(f"⚠️  No P1 signals available for new positions")

💼 PORTFOLIO CONFIGURATION
Starting Capital: $110,000
Default Position Size: 5.0% ($5,500 per position)
Max Positions: 10
Strategy Week: 1
Mode: Conservative (Building Gradually)

⚖️  Custom Allocations:
   TSLA: 50% ($55,000)
   NVDA: 10% ($11,000)

📭 No positions yet - Portfolio is 100% CASH
💰 Available: $110,000

💡 Week 1 Recommendation:
   Start with 2-3 positions
   Suggested: PLTR, TSLA, NVDA
   Total Deploy: 65.0% ($71,500)
      PLTR: 5.0% = $5,500
      TSLA: 50% = $55,000
      NVDA: 10% = $11,000


## 6.6 How to Add New Positions

**MANUAL WORKFLOW (Phase 1):**

1. **Friday:** Run this notebook to get signals
2. **Monday:** Execute trades (SELL first, then BUY)
3. **Monday Evening:** Manually update `data/portfolio_positions.csv`

**Example Entry:**
```
TSLA,2026-01-20,450.00,17,P1,P1,🟡 BUY
```

**Fields:**
- `Ticker`: Stock symbol
- `Entry_Date`: YYYY-MM-DD format
- `Entry_Price`: Your fill price
- `Shares`: Number of shares purchased
- `Entry_State`: STRONG_BUY (the state when you bought)
- `Current_State`: STRONG_BUY (will auto-update next week)
- `Entry_Signal`: 🟡 BUY

Next Friday, the scanner will automatically update Current_State and calculate P/L!

In [12]:
"""
Universe Health Check & Re-Optimization Alerts
Automatically flag conditions that suggest universe needs refresh
"""

from datetime import datetime, timedelta

print("\n" + "="*80)
print("🔍 UNIVERSE HEALTH CHECK - Re-Optimization Alerts")
print("="*80)

# Initialize alert tracking
reopt_alerts = []
alert_severity = "GREEN"  # GREEN, YELLOW, RED

# --- CONDITION 1: Universe Degradation (>30% in DOWNTREND) ---
n2_count = len(n2_signals)
total_universe = len(df_results)
n2_percentage = (n2_count / total_universe * 100) if total_universe > 0 else 0

print(f"\n📊 Condition 1: Universe Degradation")
print(f"   DOWNTREND Stocks: {n2_count}/{total_universe} ({n2_percentage:.1f}%)")

if n2_percentage > 30:
    severity = "🔴 CRITICAL"
    alert_severity = "RED"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'CRITICAL',
        'detail': f'{n2_percentage:.1f}% of universe in DOWNTREND (threshold: 30%)',
        'action': 'Re-screen S&P 500 immediately - universe is broken'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in DOWNTREND (Threshold: 30%)")
    print(f"   ⚠️  ACTION: Re-screen S&P 500 NOW - universe showing widespread weakness")
elif n2_percentage > 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Universe Degradation',
        'severity': 'WARNING',
        'detail': f'{n2_percentage:.1f}% of universe in DOWNTREND (watch threshold: 20%)',
        'action': 'Monitor closely - consider re-screening if persists 2+ weeks'
    })
    print(f"   {severity}: {n2_percentage:.1f}% in DOWNTREND (Watch at 20%)")
    print(f"   💡 Monitor: If this persists for 2+ weeks, consider re-screening")
else:
    print(f"   ✅ HEALTHY: {n2_percentage:.1f}% in N2 (Normal: <20%)")

# --- CONDITION 2: Performance Tracking (if portfolio exists) ---
print(f"\n📈 Condition 2: Portfolio Performance vs Expected")

if len(df_positions) > 0:
    # Calculate expected return based on time held
    # Expected: 56.51% annual = 1.09% per week
    expected_weekly_return = 1.09  # 56.51% / 52 weeks
    
    # Calculate actual return
    total_pl_pct_calc = ((total_value - total_cost) / total_cost * 100) if total_cost > 0 else 0
    
    # Estimate weeks held (simplified - assumes all positions entered at same time)
    weeks_held = strategy_week  # Using strategy week as proxy
    expected_return = expected_weekly_return * weeks_held
    
    performance_gap = total_pl_pct_calc - expected_return
    
    print(f"   Actual Return: {total_pl_pct_calc:+.1f}%")
    print(f"   Expected Return ({weeks_held} weeks): {expected_return:+.1f}%")
    print(f"   Performance Gap: {performance_gap:+.1f}%")
    
    if performance_gap < -10 and weeks_held > 12:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        reopt_alerts.append({
            'condition': 'Performance Lag',
            'severity': 'WARNING',
            'detail': f'Portfolio underperforming by {abs(performance_gap):.1f}% (threshold: -10%)',
            'action': 'Review stock selection - consider re-screening'
        })
        print(f"   {severity}: Underperforming by {abs(performance_gap):.1f}%")
    else:
        print(f"   ✅ ON TRACK: Performance within expected range")
else:
    print(f"   ⏳ No positions yet - tracking starts after first trades")

# --- CONDITION 3: Low Opportunity Environment ---
print(f"\n🎯 Condition 3: Market Opportunities")

p1_percentage = (len(p1_signals) / total_universe * 100) if total_universe > 0 else 0
print(f"   P1 (BUY) Signals: {len(p1_signals)}/{total_universe} ({p1_percentage:.1f}%)")

if p1_percentage < 20:
    severity = "🟡 WARNING"
    if alert_severity == "GREEN":
        alert_severity = "YELLOW"
    reopt_alerts.append({
        'condition': 'Low Opportunity',
        'severity': 'WARNING',
        'detail': f'Only {p1_percentage:.1f}% showing BUY signals',
        'action': 'Market may be bearish - check if universe needs refresh'
    })
    print(f"   {severity}: Very few opportunities")
else:
    print(f"   ✅ HEALTHY: {p1_percentage:.1f}% in P1")

# --- CONDITION 4: Time Since Last Update ---
print(f"\n📅 Condition 4: Universe Age")

portfolio_file = Path("../data/ghb_optimized_portfolio.txt")
if portfolio_file.exists():
    last_modified = datetime.fromtimestamp(portfolio_file.stat().st_mtime)
    days_since = (datetime.now() - last_modified).days
    months_since = days_since / 30.44
    
    print(f"   Last Updated: {last_modified.strftime('%Y-%m-%d')} ({months_since:.1f} months ago)")
    
    if days_since > 365:
        severity = "🔴 CRITICAL"
        alert_severity = "RED"
        reopt_alerts.append({
            'condition': 'Stale Universe',
            'severity': 'CRITICAL',
            'detail': f'Universe {months_since:.1f} months old',
            'action': 'REQUIRED: Annual re-optimization overdue'
        })
        print(f"   {severity}: Over 1 year old - RE-OPTIMIZE NOW")
    elif days_since > 180:
        severity = "🟡 WARNING"
        if alert_severity == "GREEN":
            alert_severity = "YELLOW"
        print(f"   {severity}: Over 6 months - plan update soon")
    else:
        print(f"   ✅ FRESH: Recently updated")

# --- SUMMARY ---
print(f"\n" + "="*80)
print(f"🚦 OVERALL STATUS: {alert_severity}")
print("="*80)

if alert_severity == "RED":
    print(f"🔴 CRITICAL: Re-optimization REQUIRED ({len(reopt_alerts)} issue(s))")
elif alert_severity == "YELLOW":
    print(f"🟡 WARNING: Re-optimization recommended ({len(reopt_alerts)} warning(s))")
else:
    print(f"✅ HEALTHY: No re-optimization needed")

if len(reopt_alerts) > 0:
    print(f"\n📋 ACTION ITEMS:")
    for i, alert in enumerate(reopt_alerts, 1):
        print(f"   {i}. {alert['condition']}: {alert['action']}")
    print(f"\n🔧 TO RE-OPTIMIZE: Run universe_reoptimization.ipynb")

print(f"\n" + "="*80)


🔍 UNIVERSE HEALTH CHECK - Re-Optimization Alerts

📊 Condition 1: Universe Degradation
   DOWNTREND Stocks: 1/12 (8.3%)
   ✅ HEALTHY: 8.3% in N2 (Normal: <20%)

📈 Condition 2: Portfolio Performance vs Expected
   ⏳ No positions yet - tracking starts after first trades

🎯 Condition 3: Market Opportunities
   P1 (BUY) Signals: 4/12 (33.3%)
   ✅ HEALTHY: 33.3% in P1

📅 Condition 4: Universe Age
   Last Updated: 2026-01-15 (0.0 months ago)
   ✅ FRESH: Recently updated

🚦 OVERALL STATUS: GREEN
✅ HEALTHY: No re-optimization needed



## 6.4 Universe Health Check & Re-Optimization Alerts

**Automated monitoring:** Check if universe needs refresh

## 6.7 Monday Execution Guidelines

### ⏰ TIMING STRATEGY

**9:30-10:00am: SELL N2 Positions (URGENT)**
- Execute ALL sells in first 30 minutes
- N2 = Trend broken, losses compound quickly
- Don't wait for bounce - exit fast
- Limit: Friday close - 1% (aggressive exit)

**10:00-10:30am: BUY P1 Positions (PATIENT)**
- Wait for market to settle after opening volatility
- Avoid wide spreads and overnight gap reactions
- Limit: Friday close + 1.5% (balanced entry)
- Better to miss entry than chase a 3%+ gap

### 📊 LIMIT ORDER STRATEGY

**SELL Limits (Aggressive):**
- Set at Friday close × 0.99 (1% below)
- Example: $100 Friday → Limit $99
- Priority: Get out fast, price less important
- If gaps down to $95, you exit at $95 (good!)
- If opens at $100.50, you exit at $100.50 (fine!)

**BUY Limits (Balanced):**
- Set at Friday close × 1.015 (1.5% above)
- Example: $95.50 Friday → Limit $96.93
- Gives room for normal overnight gaps
- Protects against chasing 3%+ moves
- ~90% fill rate vs 60% at exact Friday close

### 🚫 MID-WEEK TRADING RULES

**If BUY limit doesn't fill Monday:**
- ❌ **DON'T chase Tuesday-Thursday**
- ❌ **DON'T try to "catch the pullback"**
- ✅ **WAIT for next Friday's scan**
- ✅ **Enter different position that fills**

**Why wait?**
- Signals based on FRIDAY weekly close only
- Mid-week entry deviates from backtested strategy
- State might change by next Friday anyway
- ~8 trades/year means missing one is fine

**If SELL signal on Friday:**
- ✅ **MUST execute Monday 9:30am**
- ⚠️ If somehow missed, sell Tuesday morning
- ❌ **NEVER wait until next Friday**
- N2 detection already happened - act immediately

### 🎯 ASYMMETRIC STRATEGY

**Selling = Speed**
- Early execution (9:30am)
- Aggressive limits (-1%)
- No patience needed
- Preserve capital fast

**Buying = Patience**
- Delayed execution (10:00am)
- Flexible limits (+1.5%)
- Skip if too expensive
- Wait for next opportunity

### 💡 KEY PRINCIPLE

**Weekly discipline > Perfect execution**

The 34.62% CAGR backtest assumes reasonable execution at WEEKLY signals. Missing a Monday entry and waiting for next Friday is better than mid-week improvisation.

**In a hurry to stop losses, patient to enter winners.**

## 7. Export Results

Save results to CSV for record-keeping and further analysis

In [13]:
"""
GHB Portfolio Scanner - Weekly PDF Report
Generates professional PDF with market sentiment, portfolio tracking, and action items
Pure momentum strategy - no entry filters
Custom allocation: TSLA 20%, NVDA 20%, others split remaining 60%
"""

import pandas as pd
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.lib.enums import TA_LEFT, TA_CENTER
from datetime import datetime
from pathlib import Path

# Create PDF filename with timestamp
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
results_dir = Path('../ghb_scanner_results')
results_dir.mkdir(exist_ok=True)

# Archive old PDFs (limit to prevent infinite loops)
archive_dir = results_dir / 'archive'
archive_dir.mkdir(exist_ok=True)

try:
    old_pdfs = list(results_dir.glob('ghb_weekly_report_*.pdf'))
    print(f"Found {len(old_pdfs)} old PDF(s) to archive...")
    for old_file in old_pdfs[:50]:  # Limit to 50 files to prevent infinite loops
        try:
            target = archive_dir / old_file.name
            if target.exists():
                target.unlink()
            old_file.rename(target)
        except Exception as e:
            print(f"Warning: Could not archive {old_file.name}: {e}")
    print("Archiving complete.")
except Exception as e:
    print(f"Warning: Archiving failed: {e}")

pdf_file = results_dir / f'ghb_weekly_report_{datetime_str}.pdf'
doc = SimpleDocTemplate(str(pdf_file), pagesize=letter)
story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle('CustomTitle', parent=styles['Heading1'], fontSize=16, textColor=colors.HexColor('#1f4788'), spaceAfter=12)
summary_style = ParagraphStyle('Summary', parent=styles['Heading2'], fontSize=12, textColor=colors.HexColor('#1f4788'), spaceAfter=6)
subtitle_style = ParagraphStyle('Subtitle', parent=styles['Normal'], fontSize=10, textColor=colors.grey, spaceAfter=12)

# Helper functions for custom position sizing
def get_position_allocation(ticker):
    """
    Get allocation percentage for a ticker
    TSLA: 20%, NVDA: 20%, Others: split remaining 60% evenly
    """
    if ticker in position_allocations:
        return position_allocations[ticker]
    
    # Calculate remaining allocation for non-custom stocks
    total_custom_alloc = sum(position_allocations.values())
    remaining_alloc = 100 - total_custom_alloc
    
    # Split remaining among (max_positions - custom_count) stocks
    num_non_custom_slots = max_positions - len(position_allocations)
    if num_non_custom_slots > 0:
        return remaining_alloc / num_non_custom_slots
    else:
        return position_size_pct

def get_position_value(ticker):
    allocation_pct = get_position_allocation(ticker)
    return starting_cash * allocation_pct / 100

# Title Page
story.append(Paragraph("GHB Weekly Portfolio Scanner Report", title_style))
story.append(Paragraph(f"Generated: {datetime.now().strftime('%A, %B %d, %Y at %I:%M %p')}", subtitle_style))
story.append(Paragraph("<b>Price-Based Staged Entry Strategy</b>", subtitle_style))
story.append(Paragraph("<i>Custom Allocation: TSLA 20%, NVDA 20%, Others 7.5% each | Scale in at target prices</i>", subtitle_style))

# Universe Health Status
if len(reopt_alerts) > 0:
    alert_color = colors.red if alert_severity == 'RED' else colors.orange
    health_style = ParagraphStyle('HealthWarning', parent=styles['Normal'], fontSize=9, textColor=alert_color, spaceAfter=6)
    story.append(Paragraph(f"<b>Universe Health: {alert_severity}</b> - {len(reopt_alerts)} alert(s)", health_style))
else:
    health_style = ParagraphStyle('HealthGood', parent=styles['Normal'], fontSize=9, textColor=colors.green, spaceAfter=6)
    story.append(Paragraph("<b>Universe Health: All systems GREEN</b>", health_style))

story.append(Spacer(1, 0.2*inch))

# Quick Summary
pct_bullish = (len(p1_signals) / total_universe * 100) if total_universe > 0 else 0
if pct_bullish > 40:
    sentiment = "BULLISH"
elif pct_bullish > 20:
    sentiment = "NEUTRAL"
else:
    sentiment = "BEARISH"

summary_line = f"<b>Week {strategy_week}:</b> {len(p1_signals)} Buy Opportunities | {len(n2_signals)} Sell Signals | Market: {sentiment}"
story.append(Paragraph(summary_line, styles['Normal']))
story.append(Spacer(1, 0.2*inch))

# Top P1 Signals (Price-Based Staged Entry)
if len(p1_signals) > 0:
    story.append(Paragraph("Top P1 Buy Signals (Price-Based Staging)", summary_style))
    story.append(Paragraph("Initial sizing: 50% below target | 75% in target zone | 25% above target", styles['Normal']))
    story.append(Spacer(1, 0.1*inch))
    
    p1_data = [['Rank', 'Stock', 'Price', 'ROC 4W', 'RSI', 'Quality', 'Initial Fill', 'Alloc %']]
    
    for i, (_, row) in enumerate(p1_signals.head(min(10, len(p1_signals))).iterrows(), 1):
        ticker = row['Ticker']
        current = row['Close']
        roc = row['ROC_4W_%']
        rsi = row['RSI']
        quality = row.get('Entry_Quality', 'N/A')
        initial_fill = row.get('Initial_Fill_%', 100)
        alloc = get_position_allocation(ticker)
        
        # Show price-based fill percentage
        fill_display = f'{initial_fill:.0f}%'
        
        # Add target zone context if available
        if ticker in price_targets and price_targets[ticker].get('target_low'):
            target_low = price_targets[ticker]['target_low']
            target_high = price_targets[ticker]['target_high']
            if current < target_low:
                fill_display = f'{initial_fill:.0f}% (< ${target_low:.0f})'
            elif current <= target_high:
                fill_display = f'{initial_fill:.0f}% (Zone)'
            else:
                fill_display = f'{initial_fill:.0f}% (> ${target_high:.0f})'
        
        p1_data.append([
            str(i), 
            ticker, 
            f'${current:.2f}', 
            f'{roc:+.1f}%', 
            f'{rsi:.0f}',
            quality.replace('🔥 ', '').replace('✅ ', '').replace('⚠️ ', '').replace('🚨 ', ''),
            fill_display,
            f'{alloc:.0f}%'
        ])
    
    p1_table = Table(p1_data, colWidths=[0.35*inch, 0.55*inch, 0.7*inch, 0.7*inch, 0.4*inch, 0.95*inch, 1.0*inch, 0.5*inch])
    
    table_style = [
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 7),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f8f9fa')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 7),
        ('TOPPADDING', (0, 1), (-1, -1), 3),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 3),
    ]
    
    # Highlight 75% fill (in target zone)
    for i in range(1, len(p1_data)):
        if '75%' in p1_data[i][6]:
            table_style.append(('BACKGROUND', (0, i), (-1, i), colors.HexColor('#d4edda')))
            table_style.append(('FONTNAME', (1, i), (1, i), 'Helvetica-Bold'))
    
    # Highlight TSLA and NVDA if present
    for i in range(1, len(p1_data)):
        if i < len(p1_data) and p1_data[i][1] in ['TSLA', 'NVDA']:
            table_style.append(('TEXTCOLOR', (7, i), (7, i), colors.HexColor('#1f4788')))
            table_style.append(('FONTNAME', (7, i), (7, i), 'Helvetica-Bold'))
    
    p1_table.setStyle(TableStyle(table_style))
    story.append(p1_table)
    story.append(Spacer(1, 0.2*inch))

# Re-optimization alerts
if len(reopt_alerts) > 0:
    story.append(Paragraph("Universe Health Alerts", summary_style))
    for alert in reopt_alerts:
        story.append(Paragraph(f"• {alert['condition']}: {alert['detail']} - {alert['action']}", styles['Normal']))
    story.append(Spacer(1, 0.2*inch))

# Action Items for Monday
story.append(Paragraph("Action Items for Monday", summary_style))
action_items = []

# Point 1: SELL
if len(df_positions_enriched) > 0 and 'Signal' in df_positions_enriched.columns:
    n2_positions = df_positions_enriched[df_positions_enriched['State'] == 'DOWNTREND']
else:
    n2_positions = pd.DataFrame()

if len(n2_positions) > 0:
    total_sell_value = n2_positions['Position_Value'].sum()
    action_items.append(f"1. SELL ALL {len(n2_positions)} N2 position(s) - ${total_sell_value:,.0f}")
    for _, pos in n2_positions.iterrows():
        ticker = pos['Ticker']
        shares = pos['Shares']
        pl_pct = pos['P/L_%']
        pl_sign = "+" if pl_pct >= 0 else ""
        action_items.append(f"   {ticker}: SELL {shares} shares ({pl_sign}{pl_pct:.1f}%)")
else:
    if len(df_positions_enriched) > 0:
        action_items.append("1. No positions to sell - all holdings healthy")
    else:
        action_items.append("1. No current holdings - portfolio is 100% cash")

# Point 2: BUY (Price-Based Staged Entry with Custom Allocation)
if len(p1_signals) > 0:
    current_positions = len(df_positions_enriched)
    positions_to_add = min(max_positions - current_positions, len(p1_signals))
    
    # Get top P1 signals
    top_buys = p1_signals.head(positions_to_add)
    total_deploy = 0
    buy_details = []
    
    for _, row in top_buys.iterrows():
        ticker = row['Ticker']
        friday_close = row['Close']
        roc = row['ROC_4W_%']
        quality = row['Entry_Quality']
        initial_fill_pct = row.get('Initial_Fill_%', 100)
        ticker_position_value = get_position_value(ticker)
        ticker_allocation_pct = get_position_allocation(ticker)
        
        # PRICE-BASED STAGING: Initial fill based on price vs target
        shares = int((ticker_position_value * initial_fill_pct / 100) / friday_close)
        limit_price = friday_close * 1.015
        actual_cost = shares * limit_price
        total_deploy += actual_cost
        
        # Build label with target zone context
        # Always show allocation percentage (whether custom or default)
        alloc_label = f"{ticker_allocation_pct:.0f}%"
        quality_label = quality.replace('🔥 ', '').replace('✅ ', '')
        
        # Add target zone info
        target_info = ""
        if ticker in price_targets and price_targets[ticker].get('target_low'):
            target_low = price_targets[ticker]['target_low']
            target_high = price_targets[ticker]['target_high']
            if friday_close < target_low:
                target_info = f" (Below target ${target_low:.0f})"
            elif friday_close <= target_high:
                target_info = f" (In zone ${target_low:.0f}-${target_high:.0f})"
            else:
                target_info = f" (Above target ${target_high:.0f})"
        
        buy_details.append({
            'ticker': ticker,
            'alloc_label': alloc_label,
            'shares': shares,
            'limit_price': limit_price,
            'actual_cost': actual_cost,
            'roc': roc,
            'quality': quality_label,
            'initial_fill': initial_fill_pct,
            'target_info': target_info
        })
    
    if total_deploy > 0:
        action_items.append(f"2. BUY {len(buy_details)} positions (price-based sizing) - Deploy ${total_deploy:,.0f}")
        action_items.append("")  # Blank line for readability
        
        # Group by quality for better readability
        from collections import defaultdict
        grouped = defaultdict(list)
        for detail in buy_details:
            grouped[detail['quality']].append(detail)
        
        # Display each quality category
        for quality_cat in ['PULLBACK BUY', 'HEALTHY BUY', 'EXTENDED BUY']:
            if quality_cat in grouped:
                action_items.append(f"   <b>{quality_cat}:</b>")
                for detail in grouped[quality_cat]:
                    # Build allocation text - always show allocation percentage
                    alloc_text = f" ({detail['alloc_label']}), "
                    
                    # Build fill text
                    fill_pct = detail['initial_fill']
                    if fill_pct == 100:
                        fill_text = "full initial position"
                    else:
                        fill_text = f"{fill_pct:.0f}% partial fill"
                    
                    # Combine into one line
                    action_items.append(f"      • {detail['ticker']}{alloc_text}{fill_text}{detail['target_info']}, {detail['shares']} shares @ ${detail['limit_price']:.2f} = ${detail['actual_cost']:,.0f}")
                action_items.append("")  # Blank line between categories
    else:
        action_items.append("2. Portfolio full or insufficient cash")
else:
    action_items.append("2. No quality-filtered P1 buy signals - hold cash")

# Point 2.5: ADD-ONS (if any positions ready to scale in)
if len(add_on_actions) > 0:
    action_items.append(f"   ADD-ONS: {len(add_on_actions)} position(s) ready to scale in")
    for action in add_on_actions:
        action_items.append(f"      {action['Ticker']}: Add {action['Additional_Shares']} shares @ ${action['Current_Price']:.2f} ({action['Reason']})")

# Point 3: MONITOR (run once, not in a loop)
if len(df_positions_enriched) > 0:
    owned_tickers = df_positions_enriched['Ticker'].tolist()
    hold_owned = hold_signals[hold_signals['Ticker'].isin(owned_tickers)]
    if len(hold_owned) > 0:
        action_items.append(f"3. Monitor {len(hold_owned)} positions: {', '.join(hold_owned['Ticker'].tolist())}")
    else:
        action_items.append(f"3. All {len(df_positions_enriched)} positions healthy (P1)")

# Render all action items to PDF
for item in action_items:
    story.append(Paragraph(item, styles['Normal']))
    story.append(Spacer(1, 0.05*inch))

# Portfolio Holdings
story.append(Spacer(1, 0.2*inch))
story.append(Paragraph("Current Portfolio Holdings", summary_style))
story.append(Spacer(1, 0.1*inch))

if len(df_positions_enriched) > 0:
    total_value = df_positions_enriched['Position_Value'].sum()
    total_pl = df_positions_enriched['P/L_$'].sum()
    total_pl_pct = (total_pl / (total_value - total_pl) * 100) if (total_value - total_pl) > 0 else 0
    cash_remaining = starting_cash - total_value
    deployed_pct = int(total_value / starting_cash * 100)
    
    position_summaries = [
        f"<b>Total Value:</b> ${total_value:,.0f} ({deployed_pct}% deployed)",
        f"<b>Cash Remaining:</b> ${cash_remaining:,.0f}",
        f"<b>Total P/L:</b> ${total_pl:,.0f} ({total_pl_pct:+.1f}%)",
        f"<b>Positions:</b> {len(df_positions)}/{max_positions}"
    ]
    
    for summary in position_summaries:
        story.append(Paragraph(summary, styles['Normal']))
    
    story.append(Spacer(1, 0.1*inch))
    
    holdings_data = [['Ticker', 'Shares', 'Entry', 'Current', 'P/L %', 'State']]
    for _, pos in df_positions.iterrows():
        holdings_data.append([
            pos['Ticker'],
            str(pos['Shares']),
            f"${pos['Entry_Price']:.2f}",
            f"${pos['Current_Price']:.2f}",
            f"{pos['P/L_%']:+.1f}%",
            pos['Signal']
        ])
    
    holdings_table = Table(holdings_data, colWidths=[0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch, 0.8*inch])
    holdings_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#1f4788')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f0f8ff')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
        ('TOPPADDING', (0, 1), (-1, -1), 4),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 4),
    ]))
    story.append(holdings_table)
    
    if len(n2_positions) > 0:
        story.append(Spacer(1, 0.1*inch))
        alert_style = ParagraphStyle('Alert', parent=styles['Normal'], textColor=colors.red, fontSize=10)
        story.append(Paragraph(f"<b>WARNING:</b> {len(n2_positions)} position(s) in N2 - SELL Monday!", alert_style))

# Re-Optimization Alerts
if len(reopt_alerts) > 0:
    story.append(Spacer(1, 0.2*inch))

# P2/N1 Holdings (HOLD signals)
story.append(PageBreak())
story.append(Paragraph("Hold Signals (P2/N1)", summary_style))

if len(hold_signals) > 0:
    hold_data = [['Ticker', 'Price', 'D200', 'Distance', 'ROC 4W', 'State']]
    for _, row in hold_signals.iterrows():
        hold_data.append([
            row['Ticker'],
            f"${row['Close']:.2f}",
            f"${row['D200']:.2f}",
            f"{row['Distance_%']:+.1f}%",
            f"{row['ROC_4W_%']:+.1f}%",
            row['State']
        ])
    
    hold_table = Table(hold_data, colWidths=[0.7*inch, 0.85*inch, 0.85*inch, 0.85*inch, 0.85*inch, 1.0*inch])
    hold_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#6c757d')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f8f9fa')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
    ]))
    story.append(hold_table)
else:
    story.append(Paragraph("No stocks in HOLD phase (P2/N1)", styles['Normal']))

# N2 Sell Signals
story.append(Spacer(1, 0.2*inch))
story.append(Paragraph("Sell Signals (N2)", summary_style))

if len(n2_signals) > 0:
    sell_data = [['Ticker', 'Price', 'D200', 'Below D200', 'ROC 4W', 'Severity']]
    for _, row in n2_signals.iterrows():
        if row['Distance_%'] < -20:
            severity = 'SEVERE'
        elif row['Distance_%'] < -10:
            severity = 'MAJOR'
        else:
            severity = 'MINOR'
        
        sell_data.append([
            row['Ticker'],
            f"${row['Close']:.2f}",
            f"${row['D200']:.2f}",
            f"{row['Distance_%']:+.1f}%",
            f"{row['ROC_4W_%']:+.1f}%",
            severity
        ])
    
    sell_table = Table(sell_data, colWidths=[0.8*inch, 0.9*inch, 0.9*inch, 0.9*inch, 0.9*inch, 0.7*inch])
    sell_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#0056b3')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.HexColor('#f8d7da')),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 8),
    ]))
    story.append(sell_table)
else:
    story.append(Paragraph("No sell signals - all positions healthy!", styles['Normal']))

# Add Signal Glossary at bottom
story.append(Spacer(1, 0.3*inch))
story.append(Paragraph("Signal Glossary", summary_style))
story.append(Spacer(1, 0.1*inch))

glossary_style = ParagraphStyle('Glossary', parent=styles['Normal'], fontSize=9, leading=12)

glossary_text = [
    "<b>STRONG_BUY (Gold/P1):</b> Price above 200-day SMA with strong momentum (ROC &gt; 5% or Distance &gt; 10%). Action: BUY (subject to entry quality).",
    "",
    "<b>CONSOLIDATION (Gray/P2):</b> Price above 200-day SMA but momentum weak (ROC ≤ 5% and Distance ≤ 10%). Action: HOLD.",
    "",
    "<b>PULLBACK (Gray/N1):</b> Price slightly below 200-day SMA (within 5%). Could bounce back to STRONG_BUY. Action: HOLD.",
    "",
    "<b>DOWNTREND (Blue/N2):</b> Price more than 5% below 200-day SMA with weak momentum. Downtrend confirmed. Action: SELL.",
    "",
    "<b>Entry Quality (Descending Order):</b> PULLBACK BUY is STRONG_BUY + negative ROC | HEALTHY BUY (RSI &lt; 70) | EXTENDED (RSI 70-80) | OVERHEATED (Avoid)",
    "",
    "<b>Price-Based Staging:</b> Below Target = 50% fill | In Target Zone = 75% fill | Above Target = 25% fill"
]

for line in glossary_text:
    if line:
        story.append(Paragraph(line, glossary_style))
    else:
        story.append(Spacer(1, 0.05*inch))

# Build PDF
doc.build(story)
print(f"\n✅ PDF Report Generated: {pdf_file}")
print(f"   Price-Based Staged Entry - Initial sizing based on target zones")
print(f"   Custom Allocation: TSLA 20%, NVDA 20%, Others 7.5% each")
print(f"   P1 Signals: {len(p1_signals)} | N2 Signals: {len(n2_signals)}")


Found 1 old PDF(s) to archive...
Archiving complete.

✅ PDF Report Generated: ..\ghb_scanner_results\ghb_weekly_report_20260116_1759.pdf
   Price-Based Staged Entry - Initial sizing based on target zones
   Custom Allocation: TSLA 20%, NVDA 20%, Others 7.5% each
   P1 Signals: 4 | N2 Signals: 1


## 8. Export Data

Save CSV file to the same location as PDF reports

In [14]:
# Export results to CSV in same location as PDF
from pathlib import Path
from datetime import datetime

# Use same directory as PDF reports
results_dir = Path('../ghb_scanner_results')
results_dir.mkdir(exist_ok=True)

# Archive old CSV files
archive_dir = results_dir / 'archive'
archive_dir.mkdir(exist_ok=True)
for old_file in results_dir.glob('ghb_strategy_signals_*.csv'):
    target = archive_dir / old_file.name
    if target.exists():
        target.unlink()  # Delete existing file in archive
    old_file.rename(target)

# Create new CSV file
datetime_str = datetime.now().strftime('%Y%m%d_%H%M')
csv_file = results_dir / f'ghb_strategy_signals_{datetime_str}.csv'
df_results.to_csv(csv_file, index=False)

print(f"\n✅ CSV exported: {csv_file}")
print(f"   Rows: {len(df_results)} stocks")
print(f"   Location: {results_dir.absolute()}")


✅ CSV exported: ..\ghb_scanner_results\ghb_strategy_signals_20260116_1759.csv
   Rows: 12 stocks
   Location: c:\workspace\portfolio_analyser\notebooks\..\ghb_scanner_results


## GHB Strategy Quick Reference

### Your AI-Focused Portfolio
**12 stocks:** AI/Tech leaders aligned with AI dominance thesis (2023-2032)  
**Strategy:** Quality-filtered staged entry - 50% on HEALTHY BUY, 50% add-on on PULLBACK BUY  
**Custom Allocation:** TSLA 20%, NVDA 20%, Others 7.5% each (max 10 positions)

### Entry Rules (BUY)
- State = STRONG_BUY (Gold/P1)
- Price > 200-day SMA
- Strong momentum (ROC > 5% OR distance > 10%)
- **QUALITY FILTER:** Only HEALTHY BUY and PULLBACK BUY (exclude EXTENDED/OVERHEATED)
- **PRICE-BASED STAGING:**
  - Below Target: 50% initial fill
  - In Target Zone: 75% initial fill
  - Above Target: 25% initial fill
- **Entry Quality Criteria:**
  - 🔥 **PULLBACK BUY**: STRONG_BUY + negative ROC = Best opportunity
  - ✅ **HEALTHY BUY**: RSI <70, Distance <30% = Good entry
  - ⚠️ **EXTENDED**: Filtered out (avoid overextension)
  - 🚨 **OVERHEATED**: Filtered out (avoid extreme risk)

- **STRONG_BUY (Gold/P1):** Continue holding
- **CONSOLIDATION (Gray/P2):** Hold through consolidation
- **PULLBACK (Gray/N1):** Hold through shallow pullback
- **PULLBACK (Gray):** Hold through shallow pullback

- State = DOWNTREND (Blue/N2)
- Price < 200-day SMA by more than 5%
- Price < 200-day SMA
- Weak momentum
- **Exit full position**
- **Exit full position (both stages)**

### Risk Management
- Custom allocation: TSLA 20% ($22k), NVDA 20% ($22k), Others 7.5% ($8.25k)
- Staged entry: 50% initial, 50% add-on = better average prices
- Up to 10 concurrent positions (max 20 half-positions)
- **HIGH CONCENTRATION RISK:** All tech/AI exposure
- **Thesis-Dependent:** Requires AI dominance 2023-2032
- Weekly monitoring only (10-15 minutes)

### Execution Guidelines
**Monday 9:30-10:00am: SELL N2 (URGENT)**
- Execute all sells first 30 minutes (full position if staged)
- Limit: Friday close - 1% (aggressive exit)
- Don't wait for bounce - exit fast
- If missed Monday, sell Tuesday morning

- Wait for market to settle (avoid opening volatility)
- Wait for market to settle after open
- **Price-Based Sizing:** 50% below target | 75% in zone | 25% above target
- Buy only HEALTHY BUY or PULLBACK BUY quality stocks
- Custom allocation per ticker (TSLA/NVDA 20%, others 7.5%)
- If limit doesn't fill, WAIT for next Friday (don't chase mid-week)

**Key Principle:** In a hurry to stop losses, patient to enter winners

---
**Next Steps:**
1. Review signals above
2. Execute SELLS first (9:30-10:00am Monday) - full positions
3. Execute BUYS second (10:00-10:30am Monday) - 50% staging
4. Track partial vs full positions manually
5. Update portfolio CSV Monday evening
6. Run this notebook again next Friday

3. Execute BUYS second (10:00-10:30am Monday) - price-based sizing
4. Update portfolio CSV Monday evening